# Build Famous CNN Models Using `torch`

## LeNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 1. Get MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 2. Split Train Test Data is handled by torchvision datasets

100%|██████████| 9912422/9912422 [00:00<00:00, 173364320.48it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27872916.20it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 116253007.17it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 11056604.04it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# 3. Use torch to build a LeNet model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = LeNet()

# 4. Model Compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 5. Model Fit
def train(model, device, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train(model, device, train_loader, criterion, optimizer, epochs=10)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.305242
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.234400
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.086308
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.199925
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.054832
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.075595
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.023777
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.028338
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.109093
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.031632
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.029507
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.135557
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.065370
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.020308
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.073765
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.031962
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.035444
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.051980
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.016026
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.025170
Train Epoch:

In [ ]:
# 6. Model Evaluate on Test Set
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')


test(model, device, test_loader)


Test set: Average loss: 0.0000, Accuracy: 9928/10000 (99%)


## VGG16

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 1. Get CIFAR10 Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit VGG16 input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR10 normalization
])

# 2. Split Train/Test Data is already handled by the CIFAR10 dataset in torchvision
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 3. Use torch to build a VGG16 model
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 10)  # Adjusting the last layer for 10 classes of CIFAR10

# 4. Model Compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# 5. Model Fit
def train(model, device, train_loader, criterion, optimizer, epochs=10):
    model.to(device)
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 200 == 199:
                print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {running_loss / 200:.4f}')
                running_loss = 0.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(vgg16, device, train_loader, criterion, optimizer)

100%|██████████| 170498071/170498071 [00:02<00:00, 66013578.70it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 85.9MB/s]


Epoch: 1, Batch: 200, Loss: 0.7933
Epoch: 1, Batch: 400, Loss: 0.4206
Epoch: 1, Batch: 600, Loss: 0.3594


In [ ]:
# 6. Model Evaluate on Test Set
def evaluate(model, device, test_loader):
    model.eval()
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

evaluate(vgg16, device, test_loader)

## ResNet or Residual Network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
# 1. Get CIFAR10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# 3. Use torch to build a ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, 10)  # Adjust for CIFAR10's 10 classes

# 4. Model Compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# 5. Model Fit
def train(model, device, train_loader, criterion, optimizer, epochs=10):
    model.to(device)
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch+1}, Batch: {batch_idx}, Loss: {loss.item()}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(resnet, device, train_loader, criterion, optimizer)

In [ ]:
# 6. Model Evaluate on Test Set
def evaluate(model, device, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy on test set: {100 * correct / total}%')

evaluate(resnet, device, test_loader)

## DenseNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
# 1. Get CIFAR10 Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit DenseNet input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize CIFAR10 images
])

# 2. Split Train/Test Data is already handled by torchvision
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# 3. Use torch to build a DenseNet model
densenet = models.densenet121(pretrained=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 10)  # Adjust for CIFAR10 classes

# 4. Model Compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum=0.9)

# 5. Model Fit
def train(model, device, train_loader, criterion, optimizer, epochs=10):
    model.to(device)
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(densenet, device, train_loader, criterion, optimizer)

In [ ]:
# 6. Model Evaluate on Test Set
def evaluate(model, device, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy on test set: {100 * correct / total}%')

evaluate(densenet, device, test_loader)

## Inception

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 1. Get CIFAR10 Dataset
transform = transforms.Compose([
    transforms.Resize(299),  # Resize to the input size expected by Inception
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 2. Split Train/Test Data is already handled by torchvision

# 3. Build an Inception Model
inception = models.inception_v3(pretrained=True)
# Adapt the auxiliary classifier
inception.AuxLogits.fc = nn.Linear(inception.AuxLogits.fc.in_features, 10)
# Adapt the main classifier
inception.fc = nn.Linear(inception.fc.in_features, 10)

# 4. Model Compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(inception.parameters(), lr=0.001, momentum=0.9)

# 5. Model Fit
def train(model, device, train_loader, criterion, optimizer, epochs=10):
    model.to(device)
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs, aux_outputs = model(data)
            loss1 = criterion(outputs, targets)
            loss2 = criterion(aux_outputs, targets)
            loss = loss1 + 0.4*loss2
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(inception, device, train_loader, criterion, optimizer)

# 6. Model Evaluate on Test Set
def evaluate(model, device, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy on test set: {100 * correct / total}%')

evaluate(inception, device, test_loader)
